# Progress file for NER training

Install spacy in jupyter notebook env

In [17]:
import sys
!{sys.executable} -m pip install spacy

  Using cached spacy-3.2.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.2 MB)
  Using cached langcodes-3.3.0-py3-none-any.whl (181 kB)
  Using cached pydantic-1.8.2-cp38-cp38-manylinux2014_x86_64.whl (13.7 MB)
  Using cached thinc-8.0.13-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (636 kB)
  Using cached typer-0.4.0-py3-none-any.whl (27 kB)
  Using cached pathy-0.6.1-py3-none-any.whl (42 kB)
  Using cached tqdm-4.62.3-py2.py3-none-any.whl (76 kB)
  Using cached murmurhash-1.0.6-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (21 kB)
  Using cached catalogue-2.0.6-py3-none-any.whl (17 kB)
  Using cached cymem-2.0.6-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (36 kB)
  Using cached srsly-2.4.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (454 kB)
  Using cached blis-0.7.5-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (9.9 MB)
  Using cached wasabi-0.9.0-py3-none-any.whl (25 kB)

## Parsing a csv file with European addresses to spaCy readable format

In [12]:
import csv
import re
with open('address.csv', newline="\n") as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',')
    counter = 0 
    f = open("result.py", 'w')
    f.write("TRAININGS_DATA = [\n")
    f.close()
    counter = 0
    fail_numberstreet = 0
    fail_cityzip = 0 
    for row in csvreader:
        correct = True
        counter += 1
        # print(row)
        # exit()
        counter +=1
        # print("counter: " + str(counter))
        adres = ','.join(row)
        adres = adres.replace(",", ";")
        # if re.match(".*,.*,.*", adres):
            # print(adres)
            # print("FUCK YOU")
        # else:
        if not re.match(".*,.*,.*", adres):
            # print("counter: " + str(counter))
            adreslengte = len(adres)
            straat = row[0]
            totale_lengte_straat = len(straat)
            city = row[1]
            straatposbegin = ""
            straatposeind = ""
            nummerposbegin =""
            nummerposeind = ""
            cityposbegin = ""
            cityposeind = ""
            zipcodeposbegin =""
            zipcodeposeind =""
            # nummer + straat
            # oud: ([0-9]+[A-Z]*)( )?([a-zA-Z]+)
            # nieuw: (^[0-9]+.*([\u00C0-\u017Fa-zA-Z]+)),
            # probeersel: ^([0-9]+)( .*)*( )([\u00C0-\u017Fa-zA-Z]+)
            if re.match('^([0-9]+( ?bis)?) ([ \u00C0-\u017Fa-zA-Z\']+)', straat):
                split = re.search('^([0-9]+( ?bis)?) ([ \u00C0-\u017Fa-zA-Z\']+)', straat)
                splitstraat = split.group(3)
                splitnummer = split.group(1)
                straatlengte = len(splitstraat)
                nummerlengte = len(splitnummer)
                nummerposbegin = 0
                nummerposeind = nummerlengte
                straatposbegin = nummerposeind + 1
                straatposeind = straatposbegin + straatlengte

            # straat + nummer
            # oude: ([a-zA-Z]+)( )?([0-9]+)
            # nieuw: (([a-zA-Z]+) ([0-9]+.*,))
            # oude nieuwe: ^([ \u00C0-\u017Fa-zA-Z\']+) ([0-9]+( ?bis)?)
            elif re.match('^([ \u00C0-\u017Fa-zA-Z\']+) ([0-9\/]+( ?[a-zA-Z]*)?)', straat):
                split = re.search('^([ \u00C0-\u017Fa-zA-Z\']+) ([0-9\/]+( ?[a-zA-Z]*)?)', straat)
                splitstraat = split.group(1)
                splitnummer = split.group(2)
                nummerlengte = len(splitnummer)
                straatlengte = len(splitstraat)
                straatposbegin = 0
                straatposeind = straatlengte
                nummerposbegin = straatlengte + 1
                nummerposeind = nummerposbegin + nummerlengte
            else:
                fail_numberstreet +=1
                correct = False
            
            # zipcode + city
            # oud: ([A-Z]*-?[0-9]+)( )?([\u00C0-\u017Fa-zA-Z]+)
            # nieuw: ([ A-Z-]*[0-9]+) ?([\u00C0-\u017Fa-zA-Z .\/-]+)$
            if re.match('([ A-Z-]*[0-9]+) ?([\u00C0-\u017Fa-zA-Z .\/-¶¼]+)$', city):
                split = re.search('([ A-Z-]*[0-9]+) ?([\u00C0-\u017Fa-zA-Z .\/-¶¼]+)$', city)
                splitcity = split.group(2)
                splitzipcode = split.group(1)
                # print("city: " + splitcity)
                # print("zipcode: " + splitzipcode)
                citylengte = len(splitcity)
                zipcodelengte = len(splitzipcode)
                zipcodeposbegin = totale_lengte_straat + 1
                zipcodeposeind = zipcodeposbegin + zipcodelengte
                cityposbegin = zipcodeposeind + 1
                cityposeind = cityposbegin + citylengte

            # city + zipcode 
            elif re.match('([\u00C0-\u017Fa-zA-Z .\/-¶¼]+) ?([ A-Z-]*[0-9]+)$', city):
                split = re.search('([\u00C0-\u017Fa-zA-Z .\/-¶¼]+) ?([ A-Z-]*[0-9]+)$', city)
                splitcity = split.group(1)
                splitzipcode = split.group(2)
                # print("city: " + splitcity)
                # print("zipcode: " + splitzipcode)
                zipcodelengte = len(splitzipcode)
                citylengte = len(splitcity)
                cityposbegin = totale_lengte_straat + 1
                cityposeind = cityposbegin + citylengte
                zipcodeposbegin = cityposeind + 1
                zipcodeposeind = zipcodeposbegin + zipcodelengte
            else:
                fail_cityzip += 1
                correct = False




        # city = row[1]
        # citylengte = len(city)
        # straatposbegin = 0
        # straatposeind = straatlengte
        # cityposbegin = straatlengte+1
        # cityposeind = adreslengte
        # # print(row[0] + "length: " + str(len(row[0])))
            if correct:
                f = open("result.py", 'a')
                f.write("\t(\""+adres+"\","+"[("+str(straatposbegin)+","+str(straatposeind)+", \"STREET\"),("+str(nummerposbegin)+","+str(nummerposeind)+", \"NUMBER\"),("+str(cityposbegin)+","+str(cityposeind)+", \"CITY\"),("+str(zipcodeposbegin)+","+str(zipcodeposeind)+", \"ZIPCODE\")]),\n")
                f.close()
            # print("(\""+adres+"\","+"[("+str(straatposbegin)+","+str(straatposeind)+", \"STREET\"),("+str(nummerposbegin)+","+str(nummerposeind)+", \"NUMBER\"),("+str(cityposbegin)+","+str(cityposeind)+", \"CITY\"),("+str(zipcodeposbegin)+","+str(zipcodeposeind)+", \"ZIPCODE\")])")
    f = open("result.py", "a")
    f.write("]")
    f.close()
    print("failed on street: ", fail_numberstreet)
    print("failed on city: ", fail_cityzip)
        # ("Reuterstr. 131,53113 Bonn", [(0, 10, "STREET"), (11, 14, "NUMBER"), (15, 20, "ZIPCODE"), (21, 25, "CITY")],

failed on street:  71
failed on city:  616


### PrepareData.py

In [4]:
import spacy
from spacy.tokens import DocBin
from result import TRAININGS_DATA

nlp = spacy.blank("en")

infixes = nlp.Defaults.infixes + [r'\,']
nlp.tokenizer.infix_finditer = spacy.util.compile_infix_regex(infixes).finditer

db = DocBin()
for text, annotations in TRAININGS_DATA:
    doc = nlp(text)
    for token in doc:
        print(token,end='; ')
    ents = []
    for start, end, label in annotations:
        span = doc.char_span(start, end, label=label)
        if span is None:
            print("Problem:" + doc.text + " (" + doc.text[start:end] + ")" )
        else:
            ents.append(span)
    # print(ents)
    doc.ents = ents
    db.add(doc)
db.to_disk("./data/binaryData/train.spacy")

ModuleNotFoundError: No module named 'spacy'

### PrepareData.py (write to file)

In [5]:
import spacy
from spacy.tokens import DocBin
from result import TRAININGS_DATA as TD
from data.trainingData.AdressValidationData import training_data

f = open("problems.txt", 'w')
f.close()


nlp = spacy.blank("en")

infixes = nlp.Defaults.infixes + [r'\,']
nlp.tokenizer.infix_finditer = spacy.util.compile_infix_regex(infixes).finditer

# infixes = nlp.Defaults.infixes + [r'\,',]
# infix_reg = spacy.util.compile_infix_regex(infixes)
# nlp.tokenizer.infix_finditer = infix_reg.finditer

# suffixes = nlp.Defaults.suffixes + [r'(?<=[0-9]\,)',]
# suffix_regex = spacy.util.compile_suffix_regex(suffixes)
# nlp.tokenizer.suffix_search = suffix_regex.search

# prefixes = nlp.Defaults.prefixes + [r'(?<=[0-9]\,)',]
# prefix_regex = spacy.util.compile_prefix_regex(prefixes)
# nlp.tokenizer.prefix_search = prefix_regex.search


db = DocBin()
for text, annotations in TD:
    doc = nlp(text)
    # for token in doc:
        # print(token,end='; ')
    ents = []
    for start, end, label in annotations:
        span = doc.char_span(start, end, label=label)
        if span is None:
            f = open("problems.txt", 'a')
            f.write("Problem:" + doc.text + " (" + doc.text[start:end] + ")\n")
            f.close()
        else:
            ents.append(span)
    # print(ents)
    doc.ents = ents
    db.add(doc)
db.to_disk("./data/binaryData/train.spacy")


ModuleNotFoundError: No module named 'spacy'

### Run training

In [18]:
import sys
from spacy.cli.train import train

train("./config.cfg", "./output", overrides={"paths.dev": "./data/binaryData/train.spacy"})
# train("./config.cfg", "./output")

ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     57.29   19.55   12.94   39.92    0.20
  0     200        125.07   1981.01   99.16   98.94   99.39    0.99
  1     400         90.49    201.01   99.76   99.68   99.84    1.00


KeyboardInterrupt: 